In [1]:
import pandas as pd
import numpy as np
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
start_date = '2022-07-01'
final_date = '2022-07-29'
query_error_year = 2022
query_error_month = 7

# INFORMACION DE  CUBETA ERROR

In [3]:
query_error = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\POLIZAS\2022\07. JULIO\DATA\ERROR JULIO 2022.csv',
                          dtype={'report_id':object})
query_error.rename(columns ={'MAX(report_date_local)':'report_date_local'}, inplace=True)
query_error['report_date_local']= pd.to_datetime(query_error['report_date_local'])
query_error['ultimo_mov'] = pd.to_datetime(query_error['ultimo_mov'])
query_error['merchant_id_query'] = query_error['merchant_id']
query_error['ultimo_mov_date_only']= pd.to_datetime(query_error['ultimo_mov']).dt.date
query_error['day'] = query_error['ultimo_mov'].dt.day
query_error['month'] = query_error['ultimo_mov'].dt.month
query_error['year'] = query_error['ultimo_mov'].dt.year
# query_error.loc[((query_error['day'] == 30 )
#                 & (query_error['month'] == 4)
#                 & (query_error['year'] == 2022)), "mes_cierre"] = 'Actual'
# query_error.loc[(query_error['day'] == 31 )
#             & (query_error['month'] == 10 )
#             & (query_error['year'] == 2021), "mes_cierre"] = 'Actual'
query_error.loc[(query_error['month'] == query_error_month )
            & (query_error['year'] == query_error_year), "mes_cierre"] = 'Actual'
query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
0,0,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,GTS92WH86,2015-11-05,734.34,ERROR,swap devuelto,2017-08-15,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
1,1,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,8VQRDV39G,2015-11-06,597.26,ERROR,swap devuelto,2017-08-15,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
2,2,d1087b80-5383-4b90-8738-bf309a1ce297,Grupo Solutec,82G5CYSR7,2015-12-21,"9,160.77",ERROR,failed swap,2017-11-01,11,2017,ERROR,ERROR_USUAL,d1087b80-5383-4b90-8738-bf309a1ce297,2017-11-01,1,NaN
3,3,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,XWRKLC2ZL,2016-02-08,"1,216.98",ERROR,confirmar la cuenta,2017-05-12,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN
4,4,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,D9468C7W4,2016-02-16,"1,993.14",ERROR,confirmar la cuenta,2017-05-12,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN


In [4]:
query_error.shape

(93811, 17)

In [5]:
query_error.dtypes

Unnamed: 0                       int64
merchant_id                     object
merchant_name                   object
report_id                       object
report_date_local       datetime64[ns]
amount                         float64
status                          object
error_description               object
ultimo_mov              datetime64[ns]
month                            int64
year                             int64
status_cubeta                   object
esquema_query                   object
merchant_id_query               object
ultimo_mov_date_only            object
day                              int64
mes_cierre                      object
dtype: object

In [6]:
query_error['esquema_query'].unique()

array(['ERROR_USUAL', 'SIN_CLABE'], dtype=object)

In [7]:
query_error['status_cubeta'].unique()

array(['ERROR', 'NEW'], dtype=object)

# FILTRAR INFORMACION DE ACUERDO AL MES EN CURSO 

In [8]:
query_error_current_month = query_error[(query_error['mes_cierre'] == 'Actual')]
query_error_current_month.sort_values(by=['day'],inplace=True)
query_error_current_month.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_9296\3502651561.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_error_current_month.sort_values(by=['day'],inplace=True)


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
82491,82491,eb06ecba-b875-4806-bdaf-d99622d59ee4,El Taco Keto,8LVBSGH3H,2022-07-01,277.89,ERROR,Bank account information not available,2022-07-01,7,2022,NEW,SIN_CLABE,eb06ecba-b875-4806-bdaf-d99622d59ee4,2022-07-01,1,Actual
82405,82405,ffde38ac-9444-41bb-a2d5-32d9fa54ab1a,Maxilofacial,8WHLJ9ZJV,2022-07-01,210.81,ERROR,Bank account information not available,2022-07-01,7,2022,NEW,SIN_CLABE,ffde38ac-9444-41bb-a2d5-32d9fa54ab1a,2022-07-01,1,Actual
82404,82404,f29390e2-1699-4bd6-96a5-de81f9368942,Lau,RHQB2D7MM,2022-07-01,0.95,ERROR,Bank account information not available,2022-07-01,7,2022,NEW,SIN_CLABE,f29390e2-1699-4bd6-96a5-de81f9368942,2022-07-01,1,Actual
82403,82403,98443ab6-ba8b-4ee5-bd57-eddfefdfbfe2,El Roble De Panuco,N4GZ8XVVP,2022-07-01,"1,505.41",ERROR,Bank account information not available,2022-07-01,7,2022,NEW,SIN_CLABE,98443ab6-ba8b-4ee5-bd57-eddfefdfbfe2,2022-07-01,1,Actual
82402,82402,fad29aff-2483-4262-87a2-1db461b73c40,Make up yesica,4KJ3TPFTT,2022-07-01,0.95,ERROR,Bank account information not available,2022-07-01,7,2022,NEW,SIN_CLABE,fad29aff-2483-4262-87a2-1db461b73c40,2022-07-01,1,Actual


In [9]:
query_error_current_month.shape

(11703, 17)

In [10]:
query_error_current_month['month'].unique()

array([7], dtype=int64)

In [11]:
query_error_current_month.columns

Index(['Unnamed: 0', 'merchant_id', 'merchant_name', 'report_id',
       'report_date_local', 'amount', 'status', 'error_description',
       'ultimo_mov', 'month', 'year', 'status_cubeta', 'esquema_query',
       'merchant_id_query', 'ultimo_mov_date_only', 'day', 'mes_cierre'],
      dtype='object')

In [12]:
duplicates_query_error = query_error_current_month[query_error_current_month.duplicated(['report_id'] , keep=False)]
duplicates_query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre


In [13]:
duplicates_query_error.shape

(0, 17)

# PAGOS NEW DE MESES ANTERIORES

In [14]:
query_new_last_month = query_error[(query_error['mes_cierre'] != 'Actual') & (query_error['status_cubeta'] == 'NEW')]
query_new_last_month.sort_values(by=['day'],inplace=True)
query_new_last_month.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_9296\3381233170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_new_last_month.sort_values(by=['day'],inplace=True)


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
59269,59269,06fd6c7c-7c89-46d2-9cde-85c113e37c83,Genecis,2TJ6JYBBB,2022-03-03,"6,731.63",ERROR,DM,2022-06-01,6,2022,NEW,ERROR_USUAL,06fd6c7c-7c89-46d2-9cde-85c113e37c83,2022-06-01,1,NaN
61843,61843,06fd6c7c-7c89-46d2-9cde-85c113e37c83,Genecis,65Z2JZVJL,2022-03-20,"2,713.27",ERROR,DM,2022-06-01,6,2022,NEW,ERROR_USUAL,06fd6c7c-7c89-46d2-9cde-85c113e37c83,2022-06-01,1,NaN
66682,66682,06fd6c7c-7c89-46d2-9cde-85c113e37c83,Genecis,YS947SHFX,2022-04-18,"6,736.87",ERROR,DM,2022-06-01,6,2022,NEW,ERROR_USUAL,06fd6c7c-7c89-46d2-9cde-85c113e37c83,2022-06-01,1,NaN


In [15]:
query_new_last_month.shape

(3, 17)

# INFORMACIÓN QUERIES - PAGOS

In [16]:
name_header = ['merchant_id','merchant_name','user_email','report_id','report_date_local','amount','status','bank_name'
               ,'payment_date_local','bank_account_owner_name','bank_account_number','swap']

### NEXT MONTH

In [17]:
# TRADICIONAL
# trad_files_csv_2021_may = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\06. JUNE\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
# fp_files_csv_2021_may = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2021\06. JUNE\QUERIES\*.csv', recursive=False)

In [18]:
# disp_files_csv_2021_may = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
#                             .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_may])
# disp_files_csv_2021_may['date_query'] = disp_files_csv_2021_may["file_name"].str[-14:-4]
# disp_files_csv_2021_may['date_query']= pd.to_datetime(disp_files_csv_2021_may['date_query'])

In [19]:
# fp_disp_files_csv_2021_may = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
#                             .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_may])
# fp_disp_files_csv_2021_may['date_query'] = fp_disp_files_csv_2021_may["file_name"].str[-14:-4]
# fp_disp_files_csv_2021_may['date_query']= pd.to_datetime(fp_disp_files_csv_2021_may['date_query']) 

### CURRENT MONTH

In [20]:
# TRADICIONAL
trad_files_csv_2022_current = glob.glob(r'G:\My Drive\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2022\07. JULY\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2022_current = glob.glob(r'G:\My Drive\3.-Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2022\07. JULIO\QUERIES\*.csv', recursive=False)

In [21]:
disp_files_csv_2022_current = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2022_current])
disp_files_csv_2022_current['date_query'] = disp_files_csv_2022_current["file_name"].str[-14:-4]
disp_files_csv_2022_current['date_query']= pd.to_datetime(disp_files_csv_2022_current['date_query'])

In [22]:
fp_disp_files_csv_2022_current = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2022_current])
fp_disp_files_csv_2022_current['date_query'] = fp_disp_files_csv_2022_current["file_name"].str[-14:-4]
fp_disp_files_csv_2022_current['date_query']= pd.to_datetime(fp_disp_files_csv_2022_current['date_query']) 

### LAST MONTH

In [33]:
# TRADICIONAL
trad_files_csv_2021_last = glob.glob(r'G:\My Drive\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2022\06. JUNE\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2021_last = glob.glob(r'G:\My Drive\3.-Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2022\06. JUNIO\QUERIES\*.csv', recursive=False)

In [34]:
disp_files_csv_2021_last = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_last])
disp_files_csv_2021_last['date_query'] = disp_files_csv_2021_last["file_name"].str[-14:-4]
disp_files_csv_2021_last['date_query']= pd.to_datetime(disp_files_csv_2021_last['date_query'])

In [26]:
fp_disp_files_csv_2021_last = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_last])
fp_disp_files_csv_2021_last['date_query'] = fp_disp_files_csv_2021_last["file_name"].str[-14:-4]
fp_disp_files_csv_2021_last['date_query']= pd.to_datetime(fp_disp_files_csv_2021_last['date_query']) 

### SPECIAL MONTH

In [27]:
# TRADICIONAL
trad_files_csv_2021_spec = glob.glob(r'G:\My Drive\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2022\05. MAY\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2021_spec = glob.glob(r'G:\My Drive\3.-Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2022\05. MAYO\QUERIES\*.csv', recursive=False)

In [31]:
disp_files_csv_2021_spec = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_spec])
disp_files_csv_2021_spec['date_query'] = disp_files_csv_2021_spec["file_name"].str[-14:-4]
disp_files_csv_2021_spec['date_query']= pd.to_datetime(disp_files_csv_2021_spec['date_query'])

In [29]:
fp_disp_files_csv_2021_spec = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_spec])
fp_disp_files_csv_2021_spec['date_query'] = fp_disp_files_csv_2021_spec["file_name"].str[-14:-4]
fp_disp_files_csv_2021_spec['date_query']= pd.to_datetime(fp_disp_files_csv_2021_spec['date_query']) 

## ALL FILES

In [35]:
allfiles = pd.concat([disp_files_csv_2022_current,fp_disp_files_csv_2022_current
                     ,disp_files_csv_2021_last,fp_disp_files_csv_2021_last,
                     disp_files_csv_2021_spec,fp_disp_files_csv_2021_spec])
allfiles.reset_index()
allfiles.fillna('BLANK',inplace=True)
allfiles['date_query']= pd.to_datetime(allfiles['date_query'])
allfiles['report_date_local']= pd.to_datetime(allfiles['report_date_local'])
allfiles.loc[(allfiles['file_name'].str.contains('FP'),'tipo_dispersion')] = 'FP'
allfiles.loc[(allfiles['file_name'].str.contains('SWAP'),'tipo_dispersion')] = 'TRAD'
allfiles.loc[(allfiles['file_name'].str.contains('BNTE'),'tipo_dispersion')] = 'TRAD'
allfiles.loc[(allfiles['file_name'].str.contains('SANT'),'tipo_dispersion')] = 'TRAD'
allfiles.sort_values(by=['date_query','report_id'],inplace=True)
allfiles.drop(['status'],axis=1,inplace=True)
allfiles.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,disbursement_method,swap,file_name,date_query,tipo_dispersion
29552,BLANK,4f31345f-a46f-4e11-b7a7-43b6d88a8257,Farmacia San Miguel,anita_nps@hotmail.com,22273YB66,2022-04-30,130.32,"Banco Azteca, S.A.",2022-04-30 09:48:03,Ana Napoles Rodr,'127180013581467334,2022-04-30 08:07:58,BANORTE,"Ana Napoles Rodr,,130.32,127180013581467334,CL...",BNTE 2022-04-30.csv,2022-04-30,TRAD
77985,BLANK,d0d86f79-40eb-4593-9bab-dfe5d402c598,Mariscos La Carreta,rancholahacienda@outlook.com,223FC8W7R,2022-04-30,210.81,"BBVA Bancomer, S.A.",2022-04-30 09:35:37,Luis Carlos Davila Cruz,'012060028517133373,2022-04-30 08:14:07,BANORTE,"Luis Carlos Davila Cruz,,210.81,01206002851713...",BNTE 2022-04-30.csv,2022-04-30,TRAD
88961,BLANK,eedb8cae-ac9f-4e06-929f-6da3e3b2241b,ESTETICA TAPHOYAS,packotapho@gmail.com,223Q9KCRY,2022-04-30,383.30,BANAMEX,2022-04-30 09:44:50,Francisco tafoya garcia,'002180701509244542,2022-04-30 08:15:28,BANORTE,"Francisco tafoya garcia,,383.30,00218070150924...",BNTE 2022-04-30.csv,2022-04-30,TRAD
2004,BLANK,0552a54e-5617-4d61-a227-f50d9d82b033,RocketsTires,rocketstires@gmail.com,2243HLSZP,2022-04-30,862.42,"BBVA Bancomer, S.A.",2022-04-30 10:02:23,Ivan Fernando Robles Valdez,'012760014020670802,2022-04-30 08:03:45,BANORTE,"Ivan Fernando Robles Valdez,,862.42,0127600140...",BNTE 2022-04-30.csv,2022-04-30,TRAD
50324,BLANK,8721f662-3465-4568-8e57-a09ae303bcbd,Pizzas Don Juan,aleiba_saa@hotmail.com,2243T9JD5,2022-04-30,680.34,"Banco Nacional de México, S.A.",2022-04-30 10:06:39,Juan Ibanez Cortes,'002180447000535878,2022-04-30 08:10:35,BANORTE,"Juan Ibanez Cortes,,680.34,002180447000535878,...",BNTE 2022-04-30.csv,2022-04-30,TRAD


In [36]:
def tipo_pago(allfiles):
    if allfiles['date_query'] == allfiles['report_date_local']:
        return 'Pago del día'
    else:
        return 'Reprogramación'
allfiles['tipo_pago'] = allfiles.apply(tipo_pago, axis=1)

In [37]:
allfiles.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,disbursement_method,swap,file_name,date_query,tipo_dispersion,tipo_pago
29552,BLANK,4f31345f-a46f-4e11-b7a7-43b6d88a8257,Farmacia San Miguel,anita_nps@hotmail.com,22273YB66,2022-04-30,130.32,"Banco Azteca, S.A.",2022-04-30 09:48:03,Ana Napoles Rodr,'127180013581467334,2022-04-30 08:07:58,BANORTE,"Ana Napoles Rodr,,130.32,127180013581467334,CL...",BNTE 2022-04-30.csv,2022-04-30,TRAD,Pago del día
77985,BLANK,d0d86f79-40eb-4593-9bab-dfe5d402c598,Mariscos La Carreta,rancholahacienda@outlook.com,223FC8W7R,2022-04-30,210.81,"BBVA Bancomer, S.A.",2022-04-30 09:35:37,Luis Carlos Davila Cruz,'012060028517133373,2022-04-30 08:14:07,BANORTE,"Luis Carlos Davila Cruz,,210.81,01206002851713...",BNTE 2022-04-30.csv,2022-04-30,TRAD,Pago del día
88961,BLANK,eedb8cae-ac9f-4e06-929f-6da3e3b2241b,ESTETICA TAPHOYAS,packotapho@gmail.com,223Q9KCRY,2022-04-30,383.30,BANAMEX,2022-04-30 09:44:50,Francisco tafoya garcia,'002180701509244542,2022-04-30 08:15:28,BANORTE,"Francisco tafoya garcia,,383.30,00218070150924...",BNTE 2022-04-30.csv,2022-04-30,TRAD,Pago del día
2004,BLANK,0552a54e-5617-4d61-a227-f50d9d82b033,RocketsTires,rocketstires@gmail.com,2243HLSZP,2022-04-30,862.42,"BBVA Bancomer, S.A.",2022-04-30 10:02:23,Ivan Fernando Robles Valdez,'012760014020670802,2022-04-30 08:03:45,BANORTE,"Ivan Fernando Robles Valdez,,862.42,0127600140...",BNTE 2022-04-30.csv,2022-04-30,TRAD,Pago del día
50324,BLANK,8721f662-3465-4568-8e57-a09ae303bcbd,Pizzas Don Juan,aleiba_saa@hotmail.com,2243T9JD5,2022-04-30,680.34,"Banco Nacional de México, S.A.",2022-04-30 10:06:39,Juan Ibanez Cortes,'002180447000535878,2022-04-30 08:10:35,BANORTE,"Juan Ibanez Cortes,,680.34,002180447000535878,...",BNTE 2022-04-30.csv,2022-04-30,TRAD,Pago del día


In [38]:
allfiles.shape

(7977579, 18)

# CRUCE 1 - MES ACTUAL

In [39]:
cruce_1 = allfiles.merge(query_error_current_month[['report_id','amount','ultimo_mov','ultimo_mov_date_only','month','status'
                                                   ,'error_description','esquema_query','status_cubeta']],on='report_id',how='left')
cruce_1['month'].fillna('Not found',inplace=True)
cruce_1.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
0,BLANK,4f31345f-a46f-4e11-b7a7-43b6d88a8257,Farmacia San Miguel,anita_nps@hotmail.com,22273YB66,2022-04-30,130.32,"Banco Azteca, S.A.",2022-04-30 09:48:03,Ana Napoles Rodr,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1,BLANK,d0d86f79-40eb-4593-9bab-dfe5d402c598,Mariscos La Carreta,rancholahacienda@outlook.com,223FC8W7R,2022-04-30,210.81,"BBVA Bancomer, S.A.",2022-04-30 09:35:37,Luis Carlos Davila Cruz,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2,BLANK,eedb8cae-ac9f-4e06-929f-6da3e3b2241b,ESTETICA TAPHOYAS,packotapho@gmail.com,223Q9KCRY,2022-04-30,383.30,BANAMEX,2022-04-30 09:44:50,Francisco tafoya garcia,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
3,BLANK,0552a54e-5617-4d61-a227-f50d9d82b033,RocketsTires,rocketstires@gmail.com,2243HLSZP,2022-04-30,862.42,"BBVA Bancomer, S.A.",2022-04-30 10:02:23,Ivan Fernando Robles Valdez,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
4,BLANK,8721f662-3465-4568-8e57-a09ae303bcbd,Pizzas Don Juan,aleiba_saa@hotmail.com,2243T9JD5,2022-04-30,680.34,"Banco Nacional de México, S.A.",2022-04-30 10:06:39,Juan Ibanez Cortes,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [40]:
cruce_1.shape

(7977579, 26)

In [41]:
duplicates_cruce_1_previo = cruce_1[cruce_1.duplicated(['report_id'] , keep=False)]
duplicates_cruce_1_previo.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
147,BLANK,fcb7ed60-1414-4e84-a763-265119ea32f2,La Terracita,gamboah100@hotmail.com,2393HRYHV,2022-04-29,689.95,"Opciones Empresariales del Noreste, S.A. de C....",2022-04-30 16:37:01,Jorge Luis Gamboa Huerta,...,TRAD,Reprogramación,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
159,BLANK,d05a7464-04fc-4a88-bbe4-ee95f63f804b,Abarrotes Cris,hellen.db06ve@gmail.com,23DYMTGGZ,2022-04-27,130.80,"Opciones Empresariales del Noreste, S.A. de C....",2022-04-30 16:37:06,Marisol Barrios Herrera,...,TRAD,Reprogramación,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
550,BLANK,ef49748d-a9cb-449f-bdc5-694289abd3eb,Productos Helados Abieli,productosheladosabieli@gmail.com,26H82SPQP,2022-04-30,"1,570.56","Banco del Bajío, S.A.",2022-04-30 10:10:05,BLANK,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1067,BLANK,ea570490-6867-4025-9a1b-692030e097a7,Corner Wings,fatyma94@hotmail.com,2BLW98P8R,2022-04-30,"1,049.27","Opciones Empresariales del Noreste, S.A. de C....",2022-04-30 10:11:13,Fatima andrea Tzabanakis Verastegui,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1384,BLANK,d05a7464-04fc-4a88-bbe4-ee95f63f804b,Abarrotes Cris,hellen.db06ve@gmail.com,2FKPKTWWX,2022-04-22,279.79,"Opciones Empresariales del Noreste, S.A. de C....",2022-04-30 16:37:12,Marisol Barrios Herrera,...,TRAD,Reprogramación,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [42]:
duplicates_cruce_1_previo.shape

(46638, 26)

In [43]:
cruce_1_final = cruce_1[(cruce_1['month'] != 'Not found')]
cruce_1_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_1_final.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_9296\1938979547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cruce_1_final.drop_duplicates(subset='report_id',keep='last', inplace=True)


,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
2328886,BLANK,1b281894-a2f5-4a08-92a6-48cc120bf9da,H y M,kevinbros2008@gmail.com,HXNPHKQWM,2022-05-26,"23,768.98","Opciones Empresariales del Noreste, S.A. de C....",2022-05-26 12:12:08,Maria Elena Peña Pascual,...,FP,Pago del día,"23,768.98",2022-07-13,2022-07-13,7.00,ERROR,Error request by Backoffice,ERROR_USUAL,NEW
2690974,BLANK,fae70403-1c50-4323-8051-26bacfa2756d,Abarrotes Hernandez,antonio.lizbeth198522@gmail.com,HMJPYJQ9Q,2022-05-30,168.66,"Opciones Empresariales del Noreste, S.A. de C....",2022-05-30 14:17:01,Jose Antonio Hernandez Cruz,...,FP,Pago del día,168.66,2022-07-25,2022-07-25,7.00,ERROR,Error request by Backoffice,ERROR_USUAL,NEW
2710956,BLANK,fae70403-1c50-4323-8051-26bacfa2756d,Abarrotes Hernandez,antonio.lizbeth198522@gmail.com,S8896TRB2,2022-05-30,125.53,"Opciones Empresariales del Noreste, S.A. de C....",2022-05-30 16:32:27,Jose Antonio Hernandez Cruz,...,FP,Pago del día,125.53,2022-07-25,2022-07-25,7.00,ERROR,Error request by Backoffice,ERROR_USUAL,NEW
4669905,BLANK,84abc03a-1cbf-49cc-8bec-0c601cc0d152,Shop Shop,shopshop.orders@gmail.com,RCNZPFBLD,2022-06-22,"4,290.04","Opciones Empresariales del Noreste, S.A. de C....",2022-06-22 14:16:40,Levis Enriquez Martinez,...,FP,Pago del día,"4,290.04",2022-07-08,2022-07-08,7.00,ERROR,Error request by Backoffice,ERROR_USUAL,NEW
4697876,BLANK,89ece6be-8a40-4cd0-970d-97eefcb26dc7,MISC MATY,maricruzfer0202@gmail.com,49LVJP2GD,2022-06-23,336.35,"Opciones Empresariales del Noreste, S.A. de C....",2022-06-23 20:53:05,Maria cruz Ferrer Martinez,...,FP,Pago del día,336.35,2022-07-08,2022-07-08,7.00,ERROR,Error request by Backoffice,ERROR_USUAL,NEW


In [44]:
cruce_1_final.shape

(6407, 26)

In [45]:
duplicates_cruce_1_final = cruce_1_final[cruce_1_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_1_final.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta


In [46]:
duplicates_cruce_1_final.shape

(0, 26)

# CRUCE 2 MES ACTUAL

In [47]:
cruce_2 = query_error_current_month.merge(allfiles[['report_id','tipo_dispersion','tipo_pago']],on='report_id',how='left')
cruce_2['tipo_dispersion'].fillna('Not found',inplace=True)
cruce_2.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,82491,eb06ecba-b875-4806-bdaf-d99622d59ee4,El Taco Keto,8LVBSGH3H,2022-07-01,277.89,ERROR,Bank account information not available,2022-07-01,7,2022,NEW,SIN_CLABE,eb06ecba-b875-4806-bdaf-d99622d59ee4,2022-07-01,1,Actual,Not found,NaN
1,82405,ffde38ac-9444-41bb-a2d5-32d9fa54ab1a,Maxilofacial,8WHLJ9ZJV,2022-07-01,210.81,ERROR,Bank account information not available,2022-07-01,7,2022,NEW,SIN_CLABE,ffde38ac-9444-41bb-a2d5-32d9fa54ab1a,2022-07-01,1,Actual,Not found,NaN
2,82404,f29390e2-1699-4bd6-96a5-de81f9368942,Lau,RHQB2D7MM,2022-07-01,0.95,ERROR,Bank account information not available,2022-07-01,7,2022,NEW,SIN_CLABE,f29390e2-1699-4bd6-96a5-de81f9368942,2022-07-01,1,Actual,Not found,NaN
3,82403,98443ab6-ba8b-4ee5-bd57-eddfefdfbfe2,El Roble De Panuco,N4GZ8XVVP,2022-07-01,"1,505.41",ERROR,Bank account information not available,2022-07-01,7,2022,NEW,SIN_CLABE,98443ab6-ba8b-4ee5-bd57-eddfefdfbfe2,2022-07-01,1,Actual,Not found,NaN
4,82402,fad29aff-2483-4262-87a2-1db461b73c40,Make up yesica,4KJ3TPFTT,2022-07-01,0.95,ERROR,Bank account information not available,2022-07-01,7,2022,NEW,SIN_CLABE,fad29aff-2483-4262-87a2-1db461b73c40,2022-07-01,1,Actual,Not found,NaN


In [48]:
cruce_2.shape

(12084, 19)

In [49]:
cruce_2_final = cruce_2[(cruce_2['tipo_dispersion'] != 'Not found')]
#cruce_2_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_2_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
22,82382,152de79c-df2c-48df-9918-a4f43a02debd,ALUVIVARO,RMM8T7Y76,2022-07-01,0.48,ERROR,DA,2022-07-01,7,2022,NEW,ERROR_USUAL,152de79c-df2c-48df-9918-a4f43a02debd,2022-07-01,1,Actual,TRAD,Pago del día
27,82409,2858461c-cc19-40a3-b374-935e93fd3961,Mini super Los Desvelados,4MCF2Y9P3,2022-07-01,731.14,ERROR,DB,2022-07-01,7,2022,NEW,ERROR_USUAL,2858461c-cc19-40a3-b374-935e93fd3961,2022-07-01,1,Actual,TRAD,Pago del día
42,82418,b9378427-681e-489e-b429-b50611b98d2e,Arroba TOM,QYRT5FZSH,2022-07-01,159.55,ERROR,02,2022-07-01,7,2022,NEW,ERROR_USUAL,b9378427-681e-489e-b429-b50611b98d2e,2022-07-01,1,Actual,TRAD,Pago del día
45,82415,022c65ec-ab5c-4287-b9d3-3928bf92e03a,Himelda,68238H3LK,2022-07-01,39.28,ERROR,DI,2022-07-01,7,2022,NEW,ERROR_USUAL,022c65ec-ab5c-4287-b9d3-3928bf92e03a,2022-07-01,1,Actual,TRAD,Pago del día
62,82342,ab4366aa-e7a6-411c-94ae-4580da338d0e,Dentimundo,SR7W97S22,2022-07-01,"1,221.76",ERROR,R8,2022-07-01,7,2022,NEW,ERROR_USUAL,ab4366aa-e7a6-411c-94ae-4580da338d0e,2022-07-01,1,Actual,TRAD,Pago del día


In [50]:
cruce_2_final.shape

(6788, 19)

In [51]:
duplicates_cruce_2_final = cruce_2_final[cruce_2_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_2_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
769,79697,9072c0bf-3a0a-424d-be40-ecc008aa1e92,ALITAS Y PULQUES,DFDDKQD86,2022-06-19,467.62,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-07-03,7,2022,ERROR,ERROR_USUAL,9072c0bf-3a0a-424d-be40-ecc008aa1e92,2022-07-03,3,Actual,TRAD,Pago del día
770,79697,9072c0bf-3a0a-424d-be40-ecc008aa1e92,ALITAS Y PULQUES,DFDDKQD86,2022-06-19,467.62,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-07-03,7,2022,ERROR,ERROR_USUAL,9072c0bf-3a0a-424d-be40-ecc008aa1e92,2022-07-03,3,Actual,FP,Reprogramación
772,78166,9072c0bf-3a0a-424d-be40-ecc008aa1e92,ALITAS Y PULQUES,9YJ88CZDF,2022-06-11,"2,395.61",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-07-03,7,2022,ERROR,ERROR_USUAL,9072c0bf-3a0a-424d-be40-ecc008aa1e92,2022-07-03,3,Actual,TRAD,Pago del día
773,78166,9072c0bf-3a0a-424d-be40-ecc008aa1e92,ALITAS Y PULQUES,9YJ88CZDF,2022-06-11,"2,395.61",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-07-03,7,2022,ERROR,ERROR_USUAL,9072c0bf-3a0a-424d-be40-ecc008aa1e92,2022-07-03,3,Actual,FP,Reprogramación
781,74021,2e70c56f-0a5d-4056-8ada-9d1e685f9614,Carniceria Karen,L3MF2H97D,2022-05-26,793.04,ERROR,DEV.SPEIEXCEDE EL LNMITE DE SA,2022-07-03,7,2022,ERROR,ERROR_USUAL,2e70c56f-0a5d-4056-8ada-9d1e685f9614,2022-07-03,3,Actual,TRAD,Pago del día


In [52]:
duplicates_cruce_2_final.shape

(664, 19)

# AGRUPADO POR TIPO DE PAGO - MES ACTUAL

In [53]:
group_tipo_pago = pd.pivot_table(cruce_1_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_dispersion',
                                                        'tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_tipo_pago.fillna(0,inplace=True)
group_tipo_pago.head()

ultimo_mov_date_only       amount_x                 \
esquema_query                           ERROR_USUAL                  
status_cubeta                                 ERROR                  
tipo_dispersion                                  FP           TRAD   
tipo_pago                            Reprogramación Reprogramación   
0                         2022-07-01           0.00           0.00   
1                         2022-07-02           0.00           0.00   
2                         2022-07-03       8,495.32      35,001.38   
3                         2022-07-04           0.00       3,399.63   
4                         2022-07-05           0.00           0.00   

                                                                         \
esquema_query                                                             
status_cubeta            NEW                                              
tipo_dispersion           FP                        TRAD                  
tipo_pago       Pago del día Reprogramación Pago del día Reprogramación   
0                  12,016.49           0.00    26,795.38           0.00   
1                       0.00           0.00     4,234.44           0.00   
2                   1,567.68           0.00   249,748.91       4,014.92   
3                 254,364.22           0.00    77,080.92         360.29   
4                     512.65           0.00    62,841.24           0.00   

                            
esquema_query          All  
status_cubeta               
tipo_dispersion             
tipo_pago                   
0                38,811.87  
1                 4,234.44  
2               298,828.21  
3               335,205.06  
4                63,353.89

In [54]:
group_tipo_pago.shape

(30, 8)

In [55]:
group_esquema = pd.pivot_table(cruce_1_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema.fillna(0,inplace=True)
group_esquema.head()

ultimo_mov_date_only       amount_x                              \
esquema_query                         ERROR_USUAL                               
status_cubeta                               ERROR          NEW                  
tipo_pago                          Reprogramación Pago del día Reprogramación   
0                       2022-07-01           0.00    38,811.87           0.00   
1                       2022-07-02           0.00     4,234.44           0.00   
2                       2022-07-03      43,496.70   251,316.59       4,014.92   
3                       2022-07-04       3,399.63   331,445.14         360.29   
4                       2022-07-05           0.00    63,353.89           0.00   

                          
esquema_query        All  
status_cubeta             
tipo_pago                 
0              38,811.87  
1               4,234.44  
2             298,828.21  
3             335,205.06  
4              63,353.89

In [56]:
group_esquema.shape

(30, 5)

# CRUCE 3 - MES ANTERIOR

In [57]:
cruce_3 = allfiles.merge(query_new_last_month[['report_id','amount','ultimo_mov','ultimo_mov_date_only','month','status',
                                               'error_description','esquema_query','status_cubeta']],
                         on='report_id',how='left')
cruce_3['month'].fillna('Not found',inplace=True)
cruce_3.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
0,BLANK,4f31345f-a46f-4e11-b7a7-43b6d88a8257,Farmacia San Miguel,anita_nps@hotmail.com,22273YB66,2022-04-30,130.32,"Banco Azteca, S.A.",2022-04-30 09:48:03,Ana Napoles Rodr,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1,BLANK,d0d86f79-40eb-4593-9bab-dfe5d402c598,Mariscos La Carreta,rancholahacienda@outlook.com,223FC8W7R,2022-04-30,210.81,"BBVA Bancomer, S.A.",2022-04-30 09:35:37,Luis Carlos Davila Cruz,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2,BLANK,eedb8cae-ac9f-4e06-929f-6da3e3b2241b,ESTETICA TAPHOYAS,packotapho@gmail.com,223Q9KCRY,2022-04-30,383.30,BANAMEX,2022-04-30 09:44:50,Francisco tafoya garcia,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
3,BLANK,0552a54e-5617-4d61-a227-f50d9d82b033,RocketsTires,rocketstires@gmail.com,2243HLSZP,2022-04-30,862.42,"BBVA Bancomer, S.A.",2022-04-30 10:02:23,Ivan Fernando Robles Valdez,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
4,BLANK,8721f662-3465-4568-8e57-a09ae303bcbd,Pizzas Don Juan,aleiba_saa@hotmail.com,2243T9JD5,2022-04-30,680.34,"Banco Nacional de México, S.A.",2022-04-30 10:06:39,Juan Ibanez Cortes,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [58]:
cruce_3.shape

(7977579, 26)

In [59]:
cruce_3_final = cruce_3[(cruce_3['month'] != 'Not found')]
cruce_3_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_3_final.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_9296\2713612933.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cruce_3_final.drop_duplicates(subset='report_id',keep='last', inplace=True)


,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
2811441,BLANK,06fd6c7c-7c89-46d2-9cde-85c113e37c83,Genecis,paulinodejesusbermudez@gmail.com,2TJ6JYBBB,2022-03-03,"6,731.63","BBVA Bancomer, S.A.",2022-06-01 10:37:08,Paulino,...,TRAD,Reprogramación,"6,731.63",2022-06-01,2022-06-01,6.00,ERROR,DM,ERROR_USUAL,NEW
2821454,BLANK,06fd6c7c-7c89-46d2-9cde-85c113e37c83,Genecis,paulinodejesusbermudez@gmail.com,65Z2JZVJL,2022-03-20,"2,713.27","BBVA Bancomer, S.A.",2022-06-01 10:37:08,Paulino,...,TRAD,Reprogramación,"2,713.27",2022-06-01,2022-06-01,6.00,ERROR,DM,ERROR_USUAL,NEW
2892371,BLANK,06fd6c7c-7c89-46d2-9cde-85c113e37c83,Genecis,paulinodejesusbermudez@gmail.com,YS947SHFX,2022-04-18,"6,736.87","BBVA Bancomer, S.A.",2022-06-01 10:37:08,Paulino,...,TRAD,Reprogramación,"6,736.87",2022-06-01,2022-06-01,6.00,ERROR,DM,ERROR_USUAL,NEW


In [60]:
cruce_3_final.shape

(3, 26)

In [61]:
duplicates_cruce_3_final = cruce_3_final[cruce_3_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_3_final.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta


In [62]:
duplicates_cruce_3_final.shape

(0, 26)

# CRUCE 4 - MES ANTERIOR

In [63]:
cruce_4 = query_new_last_month.merge(allfiles[['report_id','tipo_dispersion','tipo_pago']],on='report_id',how='left')
cruce_4['tipo_dispersion'].fillna('Not found',inplace=True)
cruce_4.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,59269,06fd6c7c-7c89-46d2-9cde-85c113e37c83,Genecis,2TJ6JYBBB,2022-03-03,"6,731.63",ERROR,DM,2022-06-01,6,2022,NEW,ERROR_USUAL,06fd6c7c-7c89-46d2-9cde-85c113e37c83,2022-06-01,1,NaN,TRAD,Reprogramación
1,59269,06fd6c7c-7c89-46d2-9cde-85c113e37c83,Genecis,2TJ6JYBBB,2022-03-03,"6,731.63",ERROR,DM,2022-06-01,6,2022,NEW,ERROR_USUAL,06fd6c7c-7c89-46d2-9cde-85c113e37c83,2022-06-01,1,NaN,TRAD,Reprogramación
2,61843,06fd6c7c-7c89-46d2-9cde-85c113e37c83,Genecis,65Z2JZVJL,2022-03-20,"2,713.27",ERROR,DM,2022-06-01,6,2022,NEW,ERROR_USUAL,06fd6c7c-7c89-46d2-9cde-85c113e37c83,2022-06-01,1,NaN,TRAD,Reprogramación
3,61843,06fd6c7c-7c89-46d2-9cde-85c113e37c83,Genecis,65Z2JZVJL,2022-03-20,"2,713.27",ERROR,DM,2022-06-01,6,2022,NEW,ERROR_USUAL,06fd6c7c-7c89-46d2-9cde-85c113e37c83,2022-06-01,1,NaN,TRAD,Reprogramación
4,66682,06fd6c7c-7c89-46d2-9cde-85c113e37c83,Genecis,YS947SHFX,2022-04-18,"6,736.87",ERROR,DM,2022-06-01,6,2022,NEW,ERROR_USUAL,06fd6c7c-7c89-46d2-9cde-85c113e37c83,2022-06-01,1,NaN,TRAD,Reprogramación


In [64]:
cruce_4.shape

(6, 19)

In [65]:
cruce_4_final = cruce_4[(cruce_4['tipo_dispersion'] != 'Not found')]
#cruce_2_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_4_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,59269,06fd6c7c-7c89-46d2-9cde-85c113e37c83,Genecis,2TJ6JYBBB,2022-03-03,"6,731.63",ERROR,DM,2022-06-01,6,2022,NEW,ERROR_USUAL,06fd6c7c-7c89-46d2-9cde-85c113e37c83,2022-06-01,1,NaN,TRAD,Reprogramación
1,59269,06fd6c7c-7c89-46d2-9cde-85c113e37c83,Genecis,2TJ6JYBBB,2022-03-03,"6,731.63",ERROR,DM,2022-06-01,6,2022,NEW,ERROR_USUAL,06fd6c7c-7c89-46d2-9cde-85c113e37c83,2022-06-01,1,NaN,TRAD,Reprogramación
2,61843,06fd6c7c-7c89-46d2-9cde-85c113e37c83,Genecis,65Z2JZVJL,2022-03-20,"2,713.27",ERROR,DM,2022-06-01,6,2022,NEW,ERROR_USUAL,06fd6c7c-7c89-46d2-9cde-85c113e37c83,2022-06-01,1,NaN,TRAD,Reprogramación
3,61843,06fd6c7c-7c89-46d2-9cde-85c113e37c83,Genecis,65Z2JZVJL,2022-03-20,"2,713.27",ERROR,DM,2022-06-01,6,2022,NEW,ERROR_USUAL,06fd6c7c-7c89-46d2-9cde-85c113e37c83,2022-06-01,1,NaN,TRAD,Reprogramación
4,66682,06fd6c7c-7c89-46d2-9cde-85c113e37c83,Genecis,YS947SHFX,2022-04-18,"6,736.87",ERROR,DM,2022-06-01,6,2022,NEW,ERROR_USUAL,06fd6c7c-7c89-46d2-9cde-85c113e37c83,2022-06-01,1,NaN,TRAD,Reprogramación


In [66]:
cruce_4_final.shape

(6, 19)

In [67]:
duplicates_cruce_4_final = cruce_4_final[cruce_4_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_4_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,59269,06fd6c7c-7c89-46d2-9cde-85c113e37c83,Genecis,2TJ6JYBBB,2022-03-03,"6,731.63",ERROR,DM,2022-06-01,6,2022,NEW,ERROR_USUAL,06fd6c7c-7c89-46d2-9cde-85c113e37c83,2022-06-01,1,NaN,TRAD,Reprogramación
1,59269,06fd6c7c-7c89-46d2-9cde-85c113e37c83,Genecis,2TJ6JYBBB,2022-03-03,"6,731.63",ERROR,DM,2022-06-01,6,2022,NEW,ERROR_USUAL,06fd6c7c-7c89-46d2-9cde-85c113e37c83,2022-06-01,1,NaN,TRAD,Reprogramación
2,61843,06fd6c7c-7c89-46d2-9cde-85c113e37c83,Genecis,65Z2JZVJL,2022-03-20,"2,713.27",ERROR,DM,2022-06-01,6,2022,NEW,ERROR_USUAL,06fd6c7c-7c89-46d2-9cde-85c113e37c83,2022-06-01,1,NaN,TRAD,Reprogramación
3,61843,06fd6c7c-7c89-46d2-9cde-85c113e37c83,Genecis,65Z2JZVJL,2022-03-20,"2,713.27",ERROR,DM,2022-06-01,6,2022,NEW,ERROR_USUAL,06fd6c7c-7c89-46d2-9cde-85c113e37c83,2022-06-01,1,NaN,TRAD,Reprogramación
4,66682,06fd6c7c-7c89-46d2-9cde-85c113e37c83,Genecis,YS947SHFX,2022-04-18,"6,736.87",ERROR,DM,2022-06-01,6,2022,NEW,ERROR_USUAL,06fd6c7c-7c89-46d2-9cde-85c113e37c83,2022-06-01,1,NaN,TRAD,Reprogramación


In [68]:
duplicates_cruce_4_final.shape

(6, 19)

# AGRUPADO POR TIPO DE PAGO - MES ANTERIOR

In [69]:
group_tipo_pago_last_month = pd.pivot_table(cruce_3_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_dispersion',
                                                        'tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_tipo_pago_last_month.fillna(0,inplace=True)
group_tipo_pago_last_month.head()

ultimo_mov_date_only       amount_x          
esquema_query                           ERROR_USUAL       All
status_cubeta                                   NEW          
tipo_dispersion                                TRAD          
tipo_pago                            Reprogramación          
0                         2022-06-01      16,181.77 16,181.77
1                                All      16,181.77 16,181.77

In [70]:
group_tipo_pago_last_month.shape

(2, 3)

In [71]:
group_esquema_last_month = pd.pivot_table(cruce_4_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_pago'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema_last_month.fillna(0,inplace=True)
group_esquema_last_month.head()

ultimo_mov_date_only         amount          
esquema_query                         ERROR_USUAL       All
status_cubeta                                 NEW          
tipo_pago                          Reprogramación          
0                       2022-06-01      32,363.54 32,363.54
1                              All      32,363.54 32,363.54

In [72]:
group_esquema_last_month.shape

(2, 3)

In [73]:
writer = pd.ExcelWriter('FASE V. DATA POLIZA ERROR-USUAL JULIO 2022.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
group_tipo_pago.to_excel(writer, sheet_name='GRUPO TIPO PAGO')
group_esquema.to_excel(writer, sheet_name='GRUPO ESQUEMA')
group_tipo_pago_last_month.to_excel(writer, sheet_name='GRUPO TIPO PAGO-MESES ANT')
group_esquema_last_month.to_excel(writer, sheet_name='GRUPO ESQUEMA-MESES ANT')
cruce_1_final.to_excel(writer, sheet_name='DETALLE',index=False)
cruce_3_final.to_excel(writer, sheet_name='DETALLE-MESES ANTERIORES',index=False)
writer.close()